In [2]:
import pandas as pd

dir_suffix = "data/"
dir_suffix_entpackt = dir_suffix + "Daten_ausgepackt/"

pd.read_csv(dir_suffix + "tip25W_testdaten_template.csv")
df_prio = pd.read_csv(dir_suffix_entpackt + "order_products__prior.csv")
df_train = pd.read_csv(dir_suffix_entpackt + "order_products__train.csv")
df_products = pd.read_csv(dir_suffix_entpackt+"products.csv")

df_all = pd.concat([df_prio, df_train])

df_departments = pd.read_csv(dir_suffix_entpackt + "departments.csv")
df_departments = df_departments[df_departments["department_id"].between(1,5)]
df_departments

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


Aufgabe 1:
*Berechnen Sie die 5x5 Matrix M, die an Position m i,k jeweils angibt wie wahrscheinlich es ist, dass die
nachfolgende Bestellung eines Users mindestens einen Artikel aus Department k enthält
(, unter der Bedingung, dass die vorherige Bestellung desselben Users mindestens einen Artikel aus
dem Department mit ID i enthalten hat. (
Um die Matrix überschaubar zu halten, beschränkt sich die Aufgabe auf die
Departments 1 bis 5. Die wahren Wahrscheinlichkeiten sind natürlich
unbekannt; sie müssen also nur einen sinnvollen Schätzwert berechnen. Bei
Matrizen wird der erste Index nach unten, der zweite nach rechts
dargestellt.*

Also:
Wenn ein User etwas aus Department i gekauft hat: Wie wahrscheinlich ist es dann, dass er in seiner nächsten Bestellung etwas aus Department k kauft?

m i,k = P(nächste Bestellung enthält Department k | vorherige Bestellung enthielt Department i)

diese Bestellung aus k ∩ letze Bestellung aus k
_______________________________________________
letzte Bestellung aus k


Beispiel:

Für i = 3 und k = 1

Für alle user die in Prior einen Artikel aus 3 hatten und in der jetzigen Bestellung etwas aus 1??

P(A) = jemand hat aus 3 bestellt 
P(B) = jemand hat aus 1 bestellt

P(A ∩ B) = jemand hat prior aus 3 und jetzt aus 1 bestellt


| i, k    | 1 | 2 | 3 | 4 | 5 |
| -------- | ------- | ------- | ------- | ------- | ------- |
| 1        |   |   |   |   |   |
| 2        |   |   |   |   |   |
| 3        |   |   |   |   |   |
| 4        |   |   |   |   |   |
| 5        |   |   |   |   |   |

In [3]:
import pandas as pd
overview = pd.merge(df_all[:100][['order_id', 'product_id']], df_products[['product_id', 'department_id']], on="product_id", how="left")
overview.head(5)

,order_id,product_id,department_id
0,2,33120,16
1,2,28985,4
2,2,9327,13
3,2,45918,13
4,2,30035,13


In [4]:
from IPython.display import display

print("DF order Prio: ")
display(df_prio.head(1))

print("DF order train: ")
display(df_train.head(1))

print("DF orders:")
df_orders = pd.read_csv(dir_suffix_entpackt+"orders.csv")
display(df_orders.head(11))

print("DF products:")
display(df_products.head(1))

print("Departments:")
df_dep = pd.read_csv(dir_suffix_entpackt+"departments.csv")
display(df_dep.head(1))

DF order Prio: 


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1


DF order train: 


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1


DF orders:


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


DF products:


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19


Departments:


,department_id,department
0,1,frozen


In [8]:
# Sort so latest orders are first per user
df_sorted = df_orders.sort_values(['user_id', 'order_number'], ascending=[True, True])
df_sorted.head(30)
#latest_two_per_user = df_sorted.groupby('user_id').head(2).reset_index(drop=True)
#latest_two_per_user.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [9]:
df_sorted = df_sorted[["user_id", "order_id", "eval_set", "order_number"]]


df_all_small = df_all[["product_id", "order_id"]]
df_all_small

,product_id,order_id
0,33120,2
1,28985,2
2,9327,2
3,45918,2
4,30035,2
...,...,...
1384612,14233,3421063
1384613,35548,3421063
1384614,35951,3421070
1384615,16953,3421070


In [15]:
orders_merged = pd.merge(df_sorted, df_all_small, on="order_id",how="left")
orders_merged.head(30)

department_resolved = pd.merge(orders_merged[["user_id","product_id","eval_set","order_number"]],df_products[["department_id", "product_id"]], on="product_id", how="left")

department_resolved.drop(labels=["product_id"], inplace=True, axis=1)
department_resolved = department_resolved[department_resolved["department_id"].between(1,5)]
department_resolved = department_resolved.reset_index(drop=1)
department_resolved = department_resolved.sort_values(['user_id', 'order_number'], ascending=[True, True]).drop_duplicates().reset_index(drop=True)
department_resolved.head(30)


,user_id,eval_set,order_number,department_id
0,1,prior,2,4.0
1,1,prior,5,4.0
2,2,prior,1,4.0
3,2,prior,2,4.0
4,2,prior,3,4.0
5,2,prior,4,4.0
6,2,prior,4,1.0
7,2,prior,5,4.0
8,2,prior,6,1.0
9,2,prior,6,4.0


to,1,2,3,4,5
from,,,,,
1,0.338086,0.007377,0.191528,0.450028,0.012981
2,0.241941,0.115301,0.181666,0.420155,0.040938
3,0.249387,0.007182,0.284324,0.445297,0.013811
4,0.245658,0.006914,0.186867,0.549211,0.011350
5,0.184883,0.016861,0.150491,0.296526,0.351239


In [ ]:
prev = (
    df[department_resolved['order_type'] == 'prior']
    [['user_id', 'department_id']]
    .drop_duplicates()
    .rename(columns={'department_id': 'prev_dept'})
)

latest = (
    df[df['order_type'] == 'latest']
    [['user_id', 'department_id']]
    .drop_duplicates()
    .rename(columns={'department_id': 'latest_dept'})
)


In [ ]:
1. Bauen eines gesammten Datensatzes mit prior und test order_products_all

2. Produkte mit Department verknüpfen
    für jede order_id müssen wir wissen welche Products aus welchen Departments wurden bestellt.
    order_products_all mit products auf product_id joinen → jede Zeile bekommt eine deparment_id

3. Jetzt filtern wie alle orders raus die nicht aus Department 1 - 5 bestellt haben

4. jetzt haben wir eine Tabelle mit order_id, product_id, department_id (1 - 5)

5. jetzt wollen wir pro oder wissen aus welchem Department hat die jeweilige Order bestellt (mindestens eine aus 1 bis 5).
    Gruppieren nach order_id das heißt jetzt haben wir pro order die gekauften Department zuweisungen.
    ACHTUNG JETZT UNSERE ÜBERLEGUNG ... -> hier müssen wir alle duplikate rauswerfen da wir sonst mehrere Einträge pro Bestellung haben. Mehrere Produkte aus Department z.b.3 in derselben Bestellung zählen nur einmal → es geht nur darum, ob mindestens ein Artikel aus Department k drin war.

so ungefähr 

order_id	departments_in_order
10	        {2, 3}
11	        {1}
12	        {3, 5}


6. jetzt müssen wir wieder die Bestellungen mit dem User mergen da wir ja das Pro User machen müssen. 
    also mergen wir das oben nennen wir es mal order_depart mit orders (aber wir müssen alle train ignorieren) mit der order_id. Dazu nehmen wir noch das eval_set rein damit wir wissen ob es prior oder test ist.


user_id	order_number	order_id    eval_set	departments_in_order
1	    1	            2539329	    prior       {2, 4}
1	    2	            2398795	    prior       {3}
1	    3	            473747	    test        {1, 3, 5}







Formel pro intersection 




Anzahl aller Übergänge bei denen prior i war und next k
_________________________________________________________

Anzahl aller Übergänge bei denen prior k war



Beispiel



Anzahl aller Übergänge bei denen prior 3 war und next 1
_________________________________________________________

Anzahl aller Übergänge bei denen prior 3 war



wichtig ist wir dürfen für den nenner nicht nur prior und dep 3 zählen da man sonst auch alle zählt die keine nachfolgende bestellung haben es kann ja sein dass es gitb 



Da ist die Wahrscheinlichkeit dass das Übergang von i zu k passiert ist


